In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from zwad.ad.postprocess import *

from IPython.display import display, HTML
pd.set_option('display.max_rows', 2000)

# Table of contents
* [AD results comparison](#AD-results-comparison)
* [Celestial plot](#Celestial-plot)

In [3]:
data_dir = '../data/'

# Load names of fakes
df = pd.read_csv(os.path.join(data_dir, 'fakes', 'fakes_m31_fake.csv'))
fake_names = {}
for _, (oid, name) in df.iterrows():
    fake_names[oid] = name

# Load AD results

tables = {}
for alco in ['iso', 'gmm', 'lof', 'svm']:
    tables[alco] = load_ad_tables_by_patterns([os.path.join(data_dir, 'm31_{}_fake.csv'.format(alco))])
    tables[alco]['oid'] = tables[alco]['oid'].map(lambda oid: fake_names.get(oid, oid))

# Isolation Forest fakes

In [4]:
display(tables['iso'])

,oid,m31_iso_fake
0,step,-0.753090
1,ZTF18abhjrcf_format_r,-0.740498
2,Gaia16aye_3_format_r,-0.737781
3,695211400034403,-0.707322
4,695211400124577,-0.702643
5,695211400053697,-0.696436
6,695211400102351,-0.693752
7,695211400132963,-0.692504
8,ZTF18abaqxrt_format_r,-0.691396
9,695211400088968,-0.691137


# Gaussian Mixture Models fakes

In [5]:
display(tables['gmm'])

,oid,m31_gmm_fake
0,step,-1509.605536
1,695211400034403,-393.006754
2,695211200009221,-324.942218
3,695211400124577,-314.184142
4,Gaia16aye_3_format_r,-300.175128
5,695211400000352,-299.543117
6,695211400102351,-274.966990
7,695211200020939,-260.667099
8,695211400053697,-260.108951
9,695211200008801,-252.255062


# Local Outlier Factor fakes

In [6]:
display(tables['lof'])

,oid,m31_lof_fake
0,step,-8.912194
1,flat,-7.551189
2,MACHO-6.6696.60_format_R,-4.858569
3,flat_noise,-4.466071
4,Gaia16aye_2_format_r,-4.022277
5,MACHO-6.6696.60_format_B,-3.876315
6,695211200009221,-3.545021
7,695211400034403,-3.505094
8,695211100002984,-3.195999
9,Gaia16aye_3_format_r,-3.186308


# One class Support Vector Machines fakes

In [7]:
display(tables['svm'])

,oid,m31_svm_fake
0,step,1.000000
1,MACHO-6.6696.60_format_R,1.000000
2,Gaia16aye_3_format_r,1.000000
3,ZTF18abhjrcf_format_r,1.000000
4,ZTF18abaqxrt_format_r,1.000000
5,MACHO-6.6696.60_format_B,1.000000
6,flat,1.000038
7,Gaia16aye_2_format_r,1.000055
8,Gaia16aye_format_r,1.000098
9,695211200075348,1.000543
